Загрузить данные из таблицы test.xlsx.  
Проанализировать текст в колонке text, используя метод обработки текста.  
Сгенерировать вероятный краткий ответ из всего текста (состоящий из нескольких слов) и записать его в колонку answer.  

Пример:  
Текст1 - Удовлетворить  
Текст2 - Производство окончено  
Текст3 - Отказ, ИП приостановлено  
Текст4 - Отказ, невозможно установить местонахождение должника  

Такую задачу можно просто сделать через LLM, через OpenRouter с бесплатными моделями, но тут будет хромать скорость и лимиты запросов, доступность сервиса. Если же сделать через платный API, то нужны постоянные дополнительные затраты.
Так что можно сделать через более простые способы, попробовать линейные модели и бустинг модели, базовые обработки текста.

In [1]:
import pandas as pd
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

os.chdir('../')

In [2]:
df = pd.read_csv('data/processed/rule_classified.csv')

## Классификация

### Naive Bayes, Logistic Regression, SVM, RandomForest, GradientBoosting, CatBoost, XGBoost, LightGBM

Попробуем линейные модели, если они будут плохи, откинем их.  
Скорее всего бустинг модели будут хороши в этой задаче, а может и слишком сложны.  
Разница между ними под капотом большая, но можно попробовать все, сравнить результаты.

Обучим алгоритмы на тех данных, которые уже имеют верные метки

In [3]:
df_labeled = df[df['answer'].notna()].copy()
# original_indices_labeled = df_labeled.index.copy() # Для последующего объеденения с предсказаниями
df_labeled = df_labeled.reset_index(drop=True)

df_labeled['answer'].value_counts()

answer
Отказ                                             195
Частично удовлетворено                            116
Взыскание обращено                                108
Обращение рассмотрено                             100
Запрос направлен                                   78
Постановление вынесено                             36
Удовлетворено                                      27
Запрет действий                                    14
Невозможно установить местонахождение должника     10
Name: count, dtype: int64

In [ ]:
%load_ext autoreload
%autoreload 2 # Чтобы если что надо поменять в функциях в .py

from src.classification_utils import split_data, encode_labels

X_train, X_test, y_train, y_test = split_data(df_labeled)
y_train_enc, y_test_enc, le = encode_labels(y_train, y_test)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

classifiers = [
    # ('Naive Bayes', MultinomialNB()), # требует только bag-of-words или tf-idf. Имеет худший результат. Можно сразу исключить.
    ('Logistic Regression', LogisticRegression(max_iter=1000, 
                                               random_state=1, 
                                               class_weight='balanced')), # балансируем, так как слишком редкие классы
    ('SVM', LinearSVC(max_iter=2000, 
                      random_state=1, 
                      class_weight='balanced')),
    ('Decision Tree', DecisionTreeClassifier(max_depth=6,
                                             random_state=1,
                                             class_weight='balanced')),
    ('Random Forest', RandomForestClassifier(max_depth=6, # попытался ограничить глубину деревьев, так как кажется слишком сложные модели для такой задачи и идёт переобучение
                                             random_state=1,
                                             class_weight='balanced')),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=1)),
    ('CatBoost', CatBoostClassifier(verbose=0,
                                    random_state=1,
                                    n_estimators=100,
                                    max_depth=6,
                                    loss_function='MultiClass',
                                    auto_class_weights='Balanced',
                                    train_dir=None)), # всё равно сохраняет папку
    ('XGBoost', XGBClassifier(objective='multi:softmax', 
                              random_state=1, 
                              n_estimators=100,
                              learning_rate=0.1,
                              max_depth=6,
                              num_class=len(le.classes_))),
    ('LightGBM', LGBMClassifier(objective='multiclass',
                                n_estimators=100,
                                max_depth=6,
                                num_class=len(le.classes_),
                                class_weight='balanced',
                                random_state=1,
                                verbosity=-1))
]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Создаем pipeline для каждой модели
for name, classifier in classifiers:
    print(f"\n{'='*100}")
    print(f"Модель: {name}")
    print(f"{'='*100}")
    
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=1000, # Пока пробуем tf-idf
                                ngram_range=(1, 2))),
        ('classifier', classifier)
    ])
    
    pipeline.fit(X_train, y_train_enc)
    y_pred = pipeline.predict(X_test)
    y_pred_labels = le.inverse_transform(y_pred)
    
    print(classification_report(y_test, y_pred_labels, digits=3))

`UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use zero_division parameter to control this behavior. _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])`

Ошибка возникает, так как модель Naive Bayes не предсказал ни одного примера редких классов, SVM и Log Reg же попытались, хоть и не очень удачно по precision, recall, f1, но немного лучше чем просто угадывание по всем классам

SVM и Log Reg имеют лучшие показатели точности accuracy, но в этом случае это не показывает качество модели из-за несбалансированных классов.
NB имеет худшие результаты, он не определяет самые редкие классы, впрочем SVM и Log Reg методы так же имеют плохие f1 score на тех же классах, хоть и показывают хорошие macro avg (метрика, которая вместе с несбалансированными классами больше показывает модель) и weighted avg.  

Модели Random Forest, Gradient Boosting, CatBoost, XGBoost и LightGBM уже имеют намного лучшие результаты, но, кажется, есть переобучение, особенно LightGBM, так как почти все метрики 1.0 по F1, даже на редких классах. Попробовал ограничить глубину.

### Вместе с Strat K-Fold

#### Попытка 1

Попробуем проверить через K-Fold, но с Stratified, чтобы учесть несбалансированность классов и понять какие модели не нестабильны.

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for name, classifier in classifiers:
    print(f"\n{'='*100}")
    print(f"Модель: {name}")
    print(f"{'='*100}")
    
    # Создаем pipeline для каждой модели
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=1000,
                                ngram_range=(1, 2))),
        ('classifier', classifier)
    ])

    # Кросс-валидация
    y_pred_cv = cross_val_predict(
        pipeline,
        X_train,
        y_train_enc,
        cv=skf,
        method='predict'
    )

    y_pred_cv_labels = le.inverse_transform(y_pred_cv)
    y_train_labels = le.inverse_transform(y_train_enc)
    
    print(classification_report(y_train_labels, y_pred_cv_labels, digits=3))

Новые метрики уже больше похожи на правду, почти нету метрик с идеальным показателем 1.0.  
Также линейные модели лучше предсказывают самый редкий класс, чем бустинговые модели, кроме Gradient Boosting. Это может быть, потому-что бустинг модели пытаются уменьшить ошибку на всех классах, пренебрегая самым редким классом спустя множество бустинг моделей.

#### Попытка 2

- Можно попробовать увеличить размер минорного (миноритарного) класса через его дублирование (RandomOverSampler)(более простой) или через SMOTE.  
- Также можно попробовать изменить вес класса вручную для каждой модели, чем и можно заняться перед SMOTE.  
- Focal Loss для поддерживаемых моделей.
- Сравнить TF-IDF с Word2Vec, GloVe, FastText.
- Попровать сделать что-то с ruBERT.

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for name, classifier in classifiers:
    print(f"\n{'='*100}")
    print(f"Модель: {name}")
    print(f"{'='*100}")
    
    # Создаем pipeline для каждой модели
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=1000,
                                ngram_range=(1, 2))),
        ('classifier', classifier)
    ])

    # Кросс-валидация
    y_pred_cv = cross_val_predict(
        pipeline,
        X_train,
        y_train_enc,
        cv=skf,
        method='predict'
    )

    y_pred_cv_labels = le.inverse_transform(y_pred_cv)
    y_train_labels = le.inverse_transform(y_train_enc)
    
    print(classification_report(y_train_labels, y_pred_cv_labels, digits=3))

### Вместе с SMOTE и Strat K-Fold

K-Fold показал, что всё таки требуется SMOTE, так как некоторые метрики не сильно растут, а другие снижаются.

Думал аугментировать данные, но замена синонимами, перефразированием в таком датасете может не представлять сам датасет, так как фразы шаблонны.  
Можно сделать синтетические примеры через LLM, но это напоследок, если всё будет плохо с малыми классами.

In [ ]:
# Посмотрим сколько будет пример до и после
from imblearn.over_sampling import SMOTE

vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)

smote = SMOTE(random_state=1, sampling_strategy='not majority')
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_tfidf, y_train)

print("Размер классов train до:", y_train.value_counts())
print("\nРазмер классов train после:", y_train_balanced.value_counts())

print("\nРазмер train до:", len(y_train))
print("Размер train после:", len(y_train_balanced))
print("Добавлено всего:", len(y_train_balanced) - len(y_train))

In [ ]:
from imblearn.pipeline import Pipeline as ImbPipeline
# import pickle
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for name, classifier in classifiers:
    print(f"\n{'='*100}")
    print(f"Модель: {name}")
    print(f"{'='*100}")
    
    pipeline = ImbPipeline([
        ('tfidf', TfidfVectorizer(max_features=1000,
                                ngram_range=(1, 2))),
        ('smote', SMOTE(
            random_state=1,
            sampling_strategy='not majority',
            k_neighbors=4
        )),
        ('classifier', classifier)
    ])

    y_pred_cv = cross_val_predict(
        pipeline,
        X_train,
        y_train_enc,
        cv=skf,
        method='predict'
    )
    
    y_pred_cv_labels = le.inverse_transform(y_pred_cv)
    y_true_labels = le.inverse_transform(y_train_enc)
    
    print(classification_report(y_true_labels, y_pred_cv_labels, digits=3))

    # y_pred = pipeline.predict(X_test)
    # y_pred_labels = le.inverse_transform(y_pred)

    # print(classification_report(y_test, y_pred_labels))

    # trained_pipelines[name] = pipeline


    # with open(f'pipeline_{name}.pkl', 'wb') as f:
    #     pickle.dump(pipeline, f)

SMOTE помог только Naive Bayes методу предсказать ранее не предсказанные классы, и повысить все метрики.  
Метрики же на других методах не сильно поменялись или вообще стали чуть хуже. На более сложных же моделях метрики стали намного лучше 

В итоге лучшими моделями оказались Gradient Boosting, XGBoost и LightGBM.  
Можно сделать ансамбль из этих моделей

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = df_labeled['lemmatized_text']
y = df_labeled['answer']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=1,
    stratify=y
)

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

In [ ]:
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

best_model_names = ['Gradient Boosting', 'XGBoost', 'LightGBM']
best_classifiers = [pair for pair in classifiers if pair[0] in best_model_names]

trained_pipelines = {}

for name, classifier in best_classifiers:
    print(f"\n{'='*100}")
    print(f"Модель: {name}")
    print(f"{'='*100}")

    pipeline = ImbPipeline([
        ('tfidf', TfidfVectorizer(max_features=1000,
                                ngram_range=(1, 2))),
        ('smote', SMOTE(
            random_state=1,
            sampling_strategy='not majority'
        )),
        ('classifier', classifier)
    ])
    
    pipeline.fit(X_train, y_train_enc)
    
    y_pred = pipeline.predict(X_test)
    y_pred_labels = le.inverse_transform(y_pred)
    print(classification_report(y_test, y_pred_labels, digits=3))

    trained_pipelines[name] = pipeline

~~Так как метрики относительно одинаковы между Log Reg и SVM, можно выбрать одну из них. Однако Naive Bayes выдаёт иные метрики, некоторые лучше, в основном хуже. Можно попробовать ансамбль моделей~~

### Классификация неразмеченного текста

#### Через ансамбль

In [ ]:
df_unlabeled = df[df['answer'].isna()].copy()
# original_indices_unlabeled = df_labeled.index.copy()
df_unlabeled = df_unlabeled.reset_index(drop=True) # Нужно обнулить индексы, так как предсказывает только 100 из 322 почему-то
X_unlabeled = df_unlabeled['lemmatized_text']

predictions_all = {}
for name, pipeline in trained_pipelines.items():
    preds = pipeline.predict(X_unlabeled)
    predictions_all[name] = preds.ravel() if hasattr(preds, "ravel") else preds # ravel() так как CatBoost выдаёт двумерный массив, вместо одномерного

predictions_df = pd.DataFrame(predictions_all)
# # Перевернуть столбцы, чтобы из-за mode, которая при всех разных вариантов выбирает первый столбце в списке, выбиралась лучшая модель - SVM
# predictions_df = predictions_df[['SVM', 'Logistic Regression', 'Naive Bayes']]

# Преобразуем обратно в текстовые метки
for col in predictions_df.columns:
    predictions_df[col] = le.inverse_transform(predictions_df[col])

# Выбрать наиболее частый ответ среди всех строк
df_unlabeled['answer'] = predictions_df.mode(axis=1)[0]


print("Датафрейм с предсказаниями от всех моделей:")
print(predictions_df.head())

print("\nКоличество предсказанных классов:", df_unlabeled['answer'].value_counts())

In [ ]:
df_unlabeled[['answer', 'lemmatized_text']].head()

То есть любые неизвестные текста модели относят к "Обращение рассмотрено", что говорит о том, что классов недостаточно и нужно придумывать что-то иное, но, по-крайне мере модели предсказывают классы с высокими метриками

#### Итоговый датасет

In [ ]:
df_labeled.index = df[df['answer'].notna()].index
df_unlabeled.index = df[df['answer'].isna()].index

In [ ]:
df_result = pd.concat([df_labeled, df_unlabeled]).sort_index()

print(df.index)
print(df_result.index)

Индексы ~совпадают

In [ ]:
# Сохраню в xlsx, так как много строк имеют \n и \r символов, что ломает разделение строк в csv
df_result[['Id', 'text', 'answer']].to_excel('data/processed/result.xlsx', index=False)
df_result.to_excel('data/processed/result_detailed.xlsx', index=False)

## Label Propagation (не стал делать, так как того, что сверху хватило)

In [ ]:
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.preprocessing import LabelEncoder